## <strong>Import Module Bawaan Python</strong>



*   Math (Module untuk fungsi matematika di dalam python)
*   Random (Module untuk membuat angka acak di dalam python)

**Inisiasi nilai dari seed agar setiap program di running mengeluarkan hasil yang sama**

In [ ]:
import random
import math

seed = 15790490230
random.seed(seed)

## <strong>Membuat Populasi</strong>

Fungsi generate_population(length, n) berguna untuk membuat populasi yang terdiri atas kromosom berepresentasi biner (1/0).

Memiliki 2 buah parameter length dan n


1.   length merupakan panjang dari kromosom itu
2.   n merupakan jumlah kromosom di dalam populasi



In [ ]:
def generate_population(length, n):
    populasi = []
    for i in range(n):
        kromosom = []
        for j in range(length):
            kromosom.append(random.randint(0, 1))
        populasi.append(kromosom)
    
    return populasi

## <strong>Decode Kromosom</strong>

Fungsi decode(kromosom, constraint_x, constraint_y) berguna mendecode kromosom berepresentasi biner (1/0) menjadi x dan y.

Memiliki 3 buah parameter yaitu kromosom, constraint_x dan constraint_x


1.   kromosom berisi kromosom yang akan di decode
2.   constraint_x dan constraint_y merupakan batasan


In [ ]:
def decode(kromosom, constraint_x, constraint_y):
    #Titik potong kromosom
    half_kromosom = len(kromosom) 

    #Hitung bagian kromosom x
    kx = 0
    kromosom_x = kromosom[:half_kromosom]
    for i in range(1, half_kromosom + 1):
        kx = kx + kromosom_x[i-1] * (2 ** -i)

    #Hitung bagian kromosom y
    ky = 0
    kromosom_y = kromosom[half_kromosom:]
    for i in range(1, half_kromosom + 1):
        ky += kromosom_y[i-1] * (2 ** -i)

    #Hitung bagian Bawah Rumus
    lower = sum([2 ** -(i) for i in range(1, half_kromosom + 1)])

    # hitung x dan y
    x = constraint_x["min"] + (kx * (constraint_x["max"] - constraint_x["min"]) / lower)
    y = constraint_y["min"] + (ky * (constraint_y["max"] - constraint_y["min"]) / lower)

    return x, y

## <strong>Fungsi</strong>

Fungsi yang akan dicari nilai maksimumnya menggunakan Genetic Algorithm

In [ ]:
def fungsi(x,y) :
  return ((math.cos(x) * math.cos(x)) * (math.sin(y) * math.sin(y)) + (x + y))



## <strong>Fitness</strong>

Nilai kualitas untuk suatu solusi permasalahan, nilai fitness ini dari sebuah kromosom, karena kita mencari nilai maksimum maka sama dengan nilai fungsi objektifnya yaitu *f = h*

In [ ]:
def fitness(x, y):
    return fungsi(x,y)

## <strong>Seleksi Orang Tua</strong>

Seleksi orang tua dengan metode roulette wheel, berisi 2 buah parameter bernama populasi dan list_fitness

In [ ]:
def seleksi_orangtua(populasi, list_fitness):
    n = len(populasi)

    #Mendapatkan nilai minimal dan maximal dari list_fitness
    min_ = min(list_fitness)
    max_ = max(list_fitness)
    for i in range(n):
      if (max_ - min_) != 0:
        list_fitness[i] = (list_fitness[i] - min_) / (max_ - min_)
      else:
        break
        

    #Menjumlahkan semua nilai fitness
    total_fit = sum(list_fitness)

    #Roulette Whell hingga tersisa 2
    orang_tua = []
    while len(orang_tua) != 2:
        rng, i = random.random(), -1
        while rng > 0:
            i = i + 1
            rng = rng - (list_fitness[i] / total_fit)
        orang_tua.append(populasi[i])

    return orang_tua

## <strong>Crossover</strong>

Seleksi orang tua dengan metode roulette wheel, berisi 2 buah parameter bernama populasi dan list_fitness

In [ ]:
def crossover(orang_tua):

    '''
    Perkawinan silang antara 2 orang tua.

    Input:
        orang_tua : List berisi 2 kromosom.
    
    Output:
        Hasil keturunan dari 2 orang tua.
    '''

    len_krom = len(orang_tua[0])

    # pilih secara random titik perkawinan
    cross_point = random.randint(0, len_krom)

    # buat 2 anak dengan melakukan persilangan
    anak = [0, 0]
    anak[0] = orang_tua[0][:cross_point] + orang_tua[1][cross_point:]
    anak[1] = orang_tua[1][:cross_point] + orang_tua[0][cross_point:]
    
    return anak

## <strong>Mutation</strong>

Mutasi adalah proses pengubahan gen di
dalam sebuah kromosom. Proses Mutasi akan
dimulai setelah proses crossover selesai
dilakukan.Dalam program ini proses mutasi diwakili dengan fungsi mutation yang memiliki parameter anak dan prob

In [ ]:
def mutation(anak, prob):

    '''
    Melakukan mutasi dengan kemungkinan prob pada anak hasil perkawinan.

    Input:
        anak : List berisi 2 kromosom.
        prob : Kemungkinan mutasi terjadi dalam float.

    Output:
        List berisi 2 kromosom hasil mutasi.
    '''

    len_krom = len(anak[0])

    for i in range(2):
        for j in range(len_krom):
            if random.random() <= prob:
                anak[i][j] = [0, 1][ not anak[i][j] ]

    return anak

## <strong>Parameter</strong>
parameter yang digunakan pada main program nantinya

In [ ]:
# parameter
prob_crossover, prob_mutasi = 0.8, 0.01
n_pop, len_krom             = 10, 99
n_elitisme                  = 2

## <strong>Main Program</strong>
Program utama yang digunakan dalam melakukan pemecahan masalah fungsi dengan algoritma genetika

In [ ]:
#Constraint
Constraint_x = {"min" : -1, "max" : 2}
Constraint_y = {"min" : -1, "max" : 1}

gen     = 10000 # total generasi
best    = [] # fitness terbaik dalam generasi
pop     = generate_population(len_krom, n_pop) # inisial populasi
pop


for i in range(gen):

    # hitung fitness dari seluruh kromosom
    fitness_pop = []
    for kromosom in pop:
        x, y = decode(kromosom, Constraint_x, Constraint_y)
        fitness_pop.append(fitness(x, y))
    
    # cari fitness terbaik dalam populasi
    best.append( max(fitness_pop) )
    
    # sort populasi berdasarkan fitness
    sorted_pop = [k for _, k in sorted(zip(fitness_pop, pop), reverse=True)]
    
    # elitisme
    new_pop = sorted_pop[:n_elitisme]
    
    if best[i] != best[i-1]:
      print(f"Generasi terbaik pada generasi ke-{i + 1}, dengan nilai fungsi maximum: {best[i]}")
      print(f"Nilai X :", x)
      print(f"Nilai Y :", y)
  

    while len(new_pop) != n_pop:

        # lakukan seleksi orang tua pada populasi
        orang_tua = seleksi_orangtua(pop, fitness_pop)

        # lakukan crossover dan mutasi
        if random.random() < prob_crossover:
            anak = mutation(crossover(orang_tua), prob_mutasi)
        else:
            anak = orang_tua + []
        
        # bangun populasi baru
        new_pop = new_pop + anak

   
    
    pop = new_pop + []

IndexError: ignored